In [73]:
import pandas as pd
import numpy as np
import datetime

In [2]:
df = pd.read_csv(r'shedulers.csv', encoding='cp1251', sep=';')
df.head(10)

,ФИО,Расписание,Дата начала расписания,Дата окончания расписания
0,Поставщик 1,дддвсвнн,01.01.2019 0:00,10.01.2019 0:00
1,Поставщик 1,ннвннв,11.01.2019 0:00,15.01.2019 0:00
2,Поставщик 1,св,16.01.2019 0:00,20.01.2019 0:00
3,Поставщик 2,свсвсв,01.01.2019 0:00,07.01.2019 0:00
4,Поставщик 2,днвсв,08.01.2019 0:00,14.01.2019 0:00
5,Поставщик 2,ннддвсв,15.01.2019 0:00,31.12.9999 0:00
6,Поставщик 3,нвнвнв,01.01.2019 0:00,01.02.2019 0:00
7,Поставщик 3,двдвдвдв,02.02.2019 0:00,31.12.9999 0:00


Пример - есть расписания поставщиков заданное видом ДДННССВВ, где Д - дневная смена, Н - ночная смена, С - суточная смена, В - выходной

Считать что дневная смена с 08:00 по 20:00 Считать что ночная смена с 20:00 по 08:00 Считать что суточная смена с 08:00 по 08:00 завтрашнего дня

Описание работы расписания ДНСВ: 1 день - Д дневная смена 2 день - Н ночная смена 3 день - С суточная смена 4 день - В выходной день 5 день - Д дневная смена и т.д. циклично

# Задание 1
Создать таблицу (T_CONTRACTOR_SHERULER) под расписание и заполнить его с файла schedulers.csv. (Использовать любую библиотеку для заливки в СУБД (как вариант MS SQL)) Таблицу следует нормализовать.

* ID_NAME - идентификтор поставщика
* NAME - название поставщика
* SHEDULE - расписание
* DATE_BEGIN - дата начала действия расписания
* DATE_END - дата окончания действия расписания

Пример записи без нормализации:

* Поставщик 1 ДВС 01.01.2019 04.01.2019  
* Поставщик 2 НВС 05.01.2019 31.12.2019

Вводные:

* Связку полей FIO, DATE_BEGIN считать уникальной.  
* DATE_BEGIN не может привышать DATE_END.  
Можете продемонстрировать работу с ключами/ограничениями. 

``` 
create table shedulers_raw
(
    NAME varchar(256),
    SCHEDULE varchar(256),
    DATE_BEGIN varchar(256),
    DATE_END varchar(256)
);

INSERT INTO shedulers_raw
VALUES ('Поставщик 1', 'дддвсвнн', '01.01.2019 0:00', '10.01.2019 0:00'),
       ('Поставщик 1', 'ннвннв', '11.01.2019 0:00', '15.01.2019 0:00'),
       ('Поставщик 1', 'св', '16.01.2019 0:00', '20.01.2019 0:00'),
       ('Поставщик 2', 'свсвсв', '01.01.2019 0:00', '07.01.2019 0:00'),
       ('Поставщик 2', 'днвсв', '08.01.2019 0:00', '14.01.2019 0:00'),
       ('Поставщик 2', 'ннддвсв', '15.01.2019 0:00', '31.12.9999 0:00'),
       ('Поставщик 3', 'нвнвнв', '01.01.2019 0:00', '01.02.2019 0:00'),
       ('Поставщик 3', 'двдвдвдв', '02.02.2019 0:00', '31.12.9999 0:00');

create table T_CONTRACTOR_SUPPLIER
(
    ID_NAME serial primary key,
    NAME varchar(255)
);

INSERT INTO T_CONTRACTOR_SUPPLIER (NAME)
SELECT DISTINCT NAME FROM shedulers_raw;


create table T_CONTRACTOR_SCHEDULER
(
    ID_NAME int,
    SCHEDULE varchar(256),
    DATE_BEGIN date,
    DATE_END date,
    PRIMARY KEY (ID_NAME, DATE_BEGIN),
    FOREIGN KEY (ID_NAME)
        REFERENCES T_CONTRACTOR_SUPPLIER (ID_NAME),
    CHECK (DATE_BEGIN <= DATE_END)
);


insert into T_CONTRACTOR_SCHEDULER
select ID_NAME,
       SCHEDULE,
       to_date(DATE_BEGIN, 'DD.MM.YYYY'),
       to_date(DATE_END, 'DD.MM.YYYY')
from shedulers_raw LEFT JOIN T_CONTRACTOR_SUPPLIER
ON shedulers_raw.NAME = T_CONTRACTOR_SUPPLIER.NAME;

---

select *
from T_CONTRACTOR_SCHEDULER;
```

# Задание 2
Создать таблицу (T_CONTRACTOR_WORK_DAY) выходов на работу сотрудников

Таблица должна иметь следующий вид

* ID - идентификатор записи
* NAME - название поставщика
* DATE_BEGIN - Начало рабочего дня (datetime)
* DATE_END - Конец рабочего дня (datetime)

```
create table T_CONTRACTOR_WORK_DAY
(
    ID serial primary key,
    NAME varchar(256),
    DATE_BEGIN date,
    DATE_END date,
    CHECK (DATE_BEGIN <= DATE_END)
);
```

# Задание 3
Создать процедуру расчета рабочих дней.

Входящие параметры:

* Дата начала периода расчета
* Дата окончания периода расчета.
Ожидаемый результать выполнения хранимой процедуры - заполнение таблицы T_CONTRACTOR_WORK_DAY рабочими днями согласно расписания работы поставщиков из таблицы T_CONTRACTOR_SHERULER Выходные дни (В) не должны попадать в таблицу T_CONTRACTOR_WORK_DAY

Пример выполнения для Поставщика 1 (Из примера записи таблицы T_CONTRACTOR_SHERULER) с параметрами '01.01.2019' - '08.01.2019' таблица T_CONTRACTOR_WORK_DAY заполнится следующими данными:

* 1 Поставщик 1 01.01.2019 08:00 01.01.2019 20:00
* 2 Поставщик 1 03.01.2019 08:00 04.01.2019 08:00
* 3 Поставщик 1 04.01.2019 08:00 04.01.2019 08:00
* 4 Поставщик 1 05.01.2019 20:00 06.01.2019 08:00
* 5 Поставщик 1 07.01.2019 08:00 08.01.2019 08:00
* 6 Поставщик 1 08.01.2019 20:00 09.01.2019 08:00
Пояснение: для записей с 01.01.2019 по 04.01.2019 берется расписание ДВС

* 1 - Д - дневная смена далее следует выходной В - запись о выходном дне не попадает в таблицу
* 2 - С - суточная смена
* 3 - расписание закончилось, поэтому оно циклично начинается с начала (Д - дневная смена)
* 4 - С 05.01.2019 начинает действовать новое расписание - НВС берется Н - ночная смена далее следует выходной В - запись о выходном дне не попадает в таблицу
* 5 - С - суточная смена
* 6 - Снова Н - ночная смена

```
SELECT NAME, sched, DATE_BEGIN, DATE_END FROM (
SELECT NAME, regexp_split_to_table(SCHEDULE, '') sched, DATE_BEGIN, DATE_END FROM T_CONTRACTOR_SCHEDULER
    JOIN T_CONTRACTOR_SUPPLIER TCS on T_CONTRACTOR_SCHEDULER.ID_NAME = TCS.ID_NAME) q
WHERE sched <> 'в';
```

# Задание 4
* Сделать выборку содержащую сколько рабочих дней было у каждого поставщика
* Сделать выборку поставщиков, у которыйх было больше 10 рабочих дней за январь 2019 года
* Сделать выборку поставщиков, кто работал 14, 15 и 16 января 2019 года

# Задание 5
Выполните задачи 3, 4 с помощью python.

### 3

In [35]:
# заменим максимально возможной датой для пандаса
df['Дата окончания расписания'].replace('31.12.9999 0:00', '31.12.2261 0:00', inplace=True)
df['Дата начала расписания'] = pd.to_datetime(df['Дата начала расписания'], format="%d.%m.%Y %H:%M")
df['Дата окончания расписания'] = pd.to_datetime(df['Дата окончания расписания'], format="%d.%m.%Y %H:%M")

df['Расписание'] = df['Расписание'].apply(lambda x: list(x))

In [478]:
def work_day(date_begin, date_end):
    sum_total_table = pd.DataFrame()
    for person in df['ФИО'].unique():
        total_table = pd.DataFrame()
        sh2 = np.array([])
        date_begin = pd.to_datetime(date_begin)
        date_end = pd.to_datetime(date_end)
        work_table = df[df['ФИО'] == person].copy().reset_index(drop=True)

        work_table['check_begin_start'] = np.where(date_begin >= work_table['Дата начала расписания'], 1, 0)
        work_table['check_begin_end'] = np.where((date_end <= work_table['Дата окончания расписания']) & (date_end >= work_table['Дата начала расписания']), 1, 0)
        

        begin_string = work_table[work_table['check_begin_start'] == 1].index[0]
        if len(work_table[work_table['check_begin_end'] == 1]) == 0:
            end_string = work_table.index[-1]
        else:
            end_string = work_table[work_table['check_begin_end'] == 1].index[-1]
        work_table = work_table.iloc[begin_string:end_string+1]
        for i in range(len(work_table)):
            a = work_table.iloc[i]
            sh = pd.DataFrame(a[['Расписание']].explode())       
            sh.columns = [0]
            dates = pd.date_range(a['Дата начала расписания'], a['Дата окончания расписания'])
            repeat = (len(dates) // len(sh))+1
            for i in range(repeat):
                sh2 = np.hstack([np.array(sh2).reshape(-1),sh[0].values])
            sh2 = pd.DataFrame(sh2).iloc[:len(dates)]
            b = pd.DataFrame(np.vstack([sh2[0].values, dates])).T
            b[1] = pd.to_datetime(b[1])
            b['start_datetime'] = np.where((b[0]=='д') | (b[0]=='с'), b[1] + pd.to_timedelta(8, unit='h'), 
                                           np.where(b[0]=='н', b[1] + pd.to_timedelta(20, unit='h'), b[1]))
            b['end_datetime'] = np.where((b[0]=='д') | (b[0]=='н'), b['start_datetime'] + pd.to_timedelta(12, unit='h'), 
                                         np.where(b[0]=='с', b['start_datetime'] + pd.to_timedelta(24, unit='h'), b[1]))

            total_table = pd.concat([total_table, b])
        total_table['name'] = person
        sum_total_table = pd.concat([sum_total_table, total_table])
        
    sum_total_table = sum_total_table[(sum_total_table['start_datetime'] >= date_begin)]# & (sum_total_table['start_datetime'] <= date_end)]
    sum_total_table = sum_total_table[sum_total_table['end_datetime'] <= date_end + pd.to_timedelta(24, unit='h')]
    sum_total_table = sum_total_table[sum_total_table[0] != 'в']
    return sum_total_table[['name', 'start_datetime', 'end_datetime']].reset_index(drop=True)

In [479]:
work_day(date_begin='2019-01-01', date_end='2019-01-08')

,name,start_datetime,end_datetime
0,Поставщик 1,2019-01-01 08:00:00,2019-01-01 20:00:00
1,Поставщик 1,2019-01-02 08:00:00,2019-01-02 20:00:00
2,Поставщик 1,2019-01-03 08:00:00,2019-01-03 20:00:00
3,Поставщик 1,2019-01-05 08:00:00,2019-01-06 08:00:00
4,Поставщик 1,2019-01-07 20:00:00,2019-01-08 08:00:00
5,Поставщик 2,2019-01-01 08:00:00,2019-01-02 08:00:00
6,Поставщик 2,2019-01-03 08:00:00,2019-01-04 08:00:00
7,Поставщик 2,2019-01-05 08:00:00,2019-01-06 08:00:00
8,Поставщик 2,2019-01-07 08:00:00,2019-01-08 08:00:00
9,Поставщик 3,2019-01-01 20:00:00,2019-01-02 08:00:00


### 4

##### Сделать выборку содержащую сколько рабочих дней было у каждого поставщика

В задании не уточнено за какой период? Рассчитаем число рабочих дней с '2019-01-01' по настоящее время

In [491]:
work_by_now = work_day(date_begin='2019-01-01', date_end=datetime.datetime.now())

In [493]:
work_by_now.groupby('name')['start_datetime'].count()

name
Поставщик 1     16
Поставщик 2    776
Поставщик 3    545
Name: start_datetime, dtype: int64

##### Сделать выборку поставщиков, у которых было больше 10 рабочих дней за январь 2019 года

In [483]:
table_jan = work_day(date_begin='2019-01-01', date_end='2019-01-31')

In [484]:
list((table_jan.groupby('name')['start_datetime'].count() > 10).index)

['Поставщик 1', 'Поставщик 2', 'Поставщик 3']

##### Сделать выборку поставщиков, кто работал 14, 15 и 16 января 2019 года

In [480]:
table_1416 = work_day(date_begin='2019-01-14', date_end='2019-01-16')

In [481]:
list(table_1416['name'].unique())

['Поставщик 1', 'Поставщик 2', 'Поставщик 3']

In [482]:
table_1416

,name,start_datetime,end_datetime
0,Поставщик 1,2019-01-15 08:00:00,2019-01-16 08:00:00
1,Поставщик 1,2019-01-16 08:00:00,2019-01-16 20:00:00
2,Поставщик 2,2019-01-14 08:00:00,2019-01-15 08:00:00
3,Поставщик 2,2019-01-15 08:00:00,2019-01-16 08:00:00
4,Поставщик 3,2019-01-15 20:00:00,2019-01-16 08:00:00
